![Project2](Docs\Project2.JPG)

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# from langchain_community.llms.google_palm import GooglePalm
from langchain_google_genai import GoogleGenerativeAI

api_key = os.getenv('GOOGLE_API_KEY')

llm = GoogleGenerativeAI(model="gemini-pro")
# llm = GooglePalm(google_api_key=api_key, temperature=0.2)
ans = llm("Which one is best phone google pixle 9 or iphone 15 plus?")
print(ans)

D:\kworking\Aryan\GenAI\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\kworking\Aryan\GenAI\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


As of my last update in September 2023, neither the Google Pixel 9 nor the iPhone 15 Plus have been released, so I do not have any information about their specifications or features. Therefore, I cannot provide a comparison between these two devices. 

It is recommended to wait for the official release of both devices and reviews from reputable sources before making a decision on which one is best for your needs.


In [3]:
os.getcwd()

'D:\\kworking\\Aryan\\GenAI\\tutorial'

In [4]:
from langchain_community.utilities.sql_database import SQLDatabase

uri = 'sqlite:///atliq_tshirts.db'

db = SQLDatabase.from_uri(uri)
print(db.get_table_info())
print(db.get_usable_table_names())


CREATE TABLE discounts (
	discount_id INTEGER, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CHECK (pct_discount BETWEEN 0 AND 100)
)

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER, 
	brand TEXT NOT NULL, 
	color TEXT NOT NULL, 
	size TEXT NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	UNIQUE (brand, color, size), 
	CHECK (brand IN ('Van Huesen','Levi','Adidas')), 
	CHECK (color IN ('Red', 'Blue', 'Black', 'White')), 
	CHECK (size IN ('XS', 'S', 'M', 'L', 'XL')), 
	CHECK (price BETWEEN 10 AND 50)
)

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Van Huesen	White	L	33	24
3	Adidas	Black	M	29	43
8	Levi	White	XS	13	30
*/
['discounts', 't_shirts']


In [7]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
ques = db_chain.run("how many t_shirts?")



> Entering new SQLDatabaseChain chain...
how many t_shirts?
SQLQuery:```sql
SELECT COUNT(*) AS "t_shirt count"
FROM t_shirts
```

OperationalError: (sqlite3.OperationalError) near "```sql
SELECT COUNT(*) AS "t_shirt count"
FROM t_shirts
```": syntax error
[SQL: ```sql
SELECT COUNT(*) AS "t_shirt count"
FROM t_shirts
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
ques1 = db_chain("how much is the price of the inventory for all small size t_shirts?")

In [ ]:
db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'")

In [ ]:
ques2 = db_chain.run("if we sell all Large size Levi t_shirt today with discounts applied. How much revenue our store will generate(Post discount)?")

In [ ]:
query = """
SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' and size="L"
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id;
"""
db_chain.run(query)

In [ ]:
ques2 = 3346

In [ ]:
ques3 = db_chain.run("How many white color t_shirts of Levi?")

In [ ]:
ques3 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White';")

**Few Shot Learning**

In [8]:
few_shots = [
    {'Question' : "how much is the price of the inventory for all small size t_shirts?",
     'SQLQuery' : "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "10015"},
    {'Question': "if we sell all Large size Levi t_shirt today with discounts applied. How much revenue our store will generate(Post discount)?",
     'SQLQuery':"SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' and size='L' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id",
     'SQLResult': "Result of the SQL query",
     'Answer': "3346"},
    {'Question': "How many white color t_shirts of Levi?" ,
     'SQLQuery' : """SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'""",
     'SQLResult': "Result of the SQL query",
     'Answer': "97"}
]

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

D:\kworking\Aryan\GenAI\env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
D:\kworking\Aryan\GenAI\env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Create Blob of all the few_shots
to_vectorize = [" ".join(value.values()) for value in few_shots]

In [11]:
to_vectorize

["how much is the price of the inventory for all small size t_shirts? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 10015",
 "if we sell all Large size Levi t_shirt today with discounts applied. How much revenue our store will generate(Post discount)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' and size='L' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id Result of the SQL query 3346",
 "How many white color t_shirts of Levi? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' Result of the SQL query 97"]

In [12]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [13]:
vectorstore

In [14]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '97',
  'Question': 'How many white color t_shirts of Levi?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '10015',
  'Question': 'how much is the price of the inventory for all small size t_shirts?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of the SQL query'}]

In [15]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [16]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


``Setting up PromptTemplete using input variables``

In [17]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [18]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [19]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [20]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [21]:
new_chain.run('how much is the price of the inventory for all small size t_shirts?')




> Entering new SQLDatabaseChain chain...
how much is the price of the inventory for all small size t_shirts?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(4719,)]
Answer:4719
> Finished chain.


'4719'

In [22]:
new_chain.run("if we sell all Large size Levi t_shirt today with discounts applied. How much revenue our store will generate(Post discount)?")



> Entering new SQLDatabaseChain chain...
if we sell all Large size Levi t_shirt today with discounts applied. How much revenue our store will generate(Post discount)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' and size='L' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(678,)]
Answer:678
> Finished chain.


'678'